**Setting Up a New Restaurant and Utilizing Yelp's GraphQL API for Data Analysis**

Code to Connect and Authenticate Yelp Graph QL API and Some Sample Data:

In [1]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# STEP 4: Define parameters/ data we want to pull
query = """
{
  business(id: "yelp-san-francisco") {
    name
    id
    alias
    rating
    url
    location {
      city
    }
  }
}
"""

# Step 5: Send POST Request with the GraphQL query as data

response = requests.post(url, headers=headers, data=query, timeout=10)

# Step 6: Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    business = data.get('data', {}).get('business', {})

    if business:
        print("Business Name:", business['name'])
        print("Business ID:", business['id'])
        print("Business Alias:", business['alias'])
        print("Rating:", business['rating'])
        print("URL:", business['url'])
        print("Location (City):", business['location']['city'])  # Updated print statement
    else:
            print("No business found.")
else:
    # Handle Rate Limit Exceeded
    if response.status_code == 429:
        print("Error: Rate limit exceeded. Please wait and try again later.")
        rate_limit_remaining = response.headers.get('RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")
    else:
        print(f"Error: {response.status_code}")
        print("Response Text:", response.text)

Business Name: Yelp
Business ID: 4kMBvIEWPxWkWKFN__8SxQ
Business Alias: yelp-san-francisco
Rating: 2.0
URL: https://www.yelp.com/biz/yelp-san-francisco?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Location (City): San Francisco


Business Context and Some Sample Data in form of different queries from Graph QL

Query 1: Fetching Top 10 Restaurants in Seattle area limits

In [3]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Seattle", limit: 10) {
    business {
      name
      rating
      location {
        address1
        city
        state
        country
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for i, business in enumerate(businesses, start=1):
            print(f"Restaurant {i}:")
            print(f"Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print(f"Address: {business['location']['address1']}, {business['location']['city']}, {business['location']['state']}, {business['location']['country']}")
            print("=" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")

Restaurant 1:
Name: The Pink Door
Rating: 4.4
Address: 1919 Post Alley, Seattle, WA, US
Restaurant 2:
Name: Von's 1000 Spirits
Rating: 4.5
Address: 1225 1st Ave, Seattle, WA, US
Restaurant 3:
Name: Bangrak Market
Rating: 4.3
Address: 2319 2nd Ave, Seattle, WA, US
Restaurant 4:
Name: Shaker + Spear
Rating: 4.4
Address: 2000 2nd Ave, Seattle, WA, US
Restaurant 5:
Name: Reckless Noodle House
Rating: 4.5
Address: 2519 S Jackson St, Seattle, WA, US
Restaurant 6:
Name: Kedai Makan
Rating: 4.1
Address: 1449 E Pine St, Seattle, WA, US
Restaurant 7:
Name: Biang Biang Noodles
Rating: 4.3
Address: 601 E Pike St, Seattle, WA, US
Restaurant 8:
Name: Biscuit Bitch
Rating: 4.2
Address: 1909 1st Ave, Seattle, WA, US
Restaurant 9:
Name: Radiator Whiskey
Rating: 4.4
Address: 94 Pike St, Seattle, WA, US
Restaurant 10:
Name: Paju
Rating: 4.5
Address: 513 Westlake Ave N, Seattle, WA, US


There are really good amazing restuarants in Seattle area that cater needs of different cusines

Query 2: Get number of restaurant business units based on keyword Pizza in Seattle and its areas

In [4]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "pizza", location: "Seattle", limit: 50) {
    total
    business {
      name
      location {
        city
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        city_count = {}
        for business in businesses:
            city = business['location']['city']
            if city in city_count:
                city_count[city] += 1
            else:
                city_count[city] = 1

        # Display number of business units by cities
        for city, count in city_count.items():
            print(f"City: {city} | Number of Businesses: {count}")
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


City: Seattle | Number of Businesses: 47
City: Lynnwood | Number of Businesses: 1
City: Kingston | Number of Businesses: 1
City: Burien | Number of Businesses: 1


By this we can understand their are numerous Pizza serving restaurants in Seattle and its areas

Query 3: Getting Top 10 restaurants and their names based on most reviews

In [5]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Seattle", limit: 10) {
    business {
      name
      review_count
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Restaurant Name: {business['name']}")
            print(f"Review Count: {business['review_count']}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Restaurant Name: The Pink Door
Review Count: 7458
--------------------------------------------------
Restaurant Name: Von's 1000 Spirits
Review Count: 3429
--------------------------------------------------
Restaurant Name: Bangrak Market
Review Count: 1594
--------------------------------------------------
Restaurant Name: Shaker + Spear
Review Count: 1140
--------------------------------------------------
Restaurant Name: Reckless Noodle House
Review Count: 491
--------------------------------------------------
Restaurant Name: Kedai Makan
Review Count: 1452
--------------------------------------------------
Restaurant Name: Biang Biang Noodles
Review Count: 1411
--------------------------------------------------
Restaurant Name: Biscuit Bitch
Review Count: 4941
--------------------------------------------------
Restaurant Name: Radiator Whiskey
Review Count: 1439
--------------------------------------------------
Restaurant Name: Paju
Review Count: 353
------------------------------

This helps us understand the most famous Restaurants based on customer reviews count and implement some strategies they are doing in our future business as well

Query 4: Now we will get Restaurants based on different cusine types they have to offer in Seattle area

In [6]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Seattle", limit: 10) {
    business {
      name
      location {
        address1
        city
        state
        country
      }
      url
      categories {
        title
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            # Extracting relevant data
            name = business.get('name', 'N/A')
            address = business.get('location', {}).get('address1', 'N/A')
            city = business.get('location', {}).get('city', 'N/A')
            state = business.get('location', {}).get('state', 'N/A')
            country = business.get('location', {}).get('country', 'N/A')
            url = business.get('url', 'N/A')
            categories = ", ".join([category['title'] for category in business.get('categories', [])])

            # Printing the results
            print(f"Restaurant Name: {name}")
            print(f"Address: {address}, {city}, {state}, {country}")
            print(f"URL: {url}")
            print(f"Categories (Cuisines): {categories}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Restaurant Name: The Pink Door
Address: 1919 Post Alley, Seattle, WA, US
URL: https://www.yelp.com/biz/the-pink-door-seattle-4?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisines): Italian, Wine Bars, Pasta Shops
--------------------------------------------------
Restaurant Name: Von's 1000 Spirits
Address: 1225 1st Ave, Seattle, WA, US
URL: https://www.yelp.com/biz/vons-1000-spirits-seattle-4?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisines): New American, Burgers, Pubs
--------------------------------------------------
Restaurant Name: Bangrak Market
Address: 2319 2nd Ave, Seattle, WA, US
URL: https://www.yelp.com/biz/bangrak-market-seattle?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisines): Bars, Thai, T

This will give us stats and restaurants with different cusines, locations, websites they host and help us understand how each business unit varies and caters/ serves to customer needs

Query 5: Lets find some events happening in the Seattle area for Food servings or which have relation to Food in any way

In [7]:
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'  # Ensure this is correct. Yelp mostly uses REST endpoints.

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Correct content type
}

# 1. Define parameters/data for the query
category = "Food"  # Setting the category to 'Food'
location = "Seattle"
query = {
    "query": f"""
    {{
      search(term: "{category}", location: "{location}") {{
        business {{
          name
          rating
          location {{
            address1
            city
            state
            country
          }}
          categories {{
            title
          }}
        }}
      }}
    }}
    """
}

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            # Extracting relevant data
            name = business.get('name', 'N/A')
            rating = business.get('rating', 'N/A')
            location = business.get('location', {})
            address = location.get('address1', 'N/A')
            city = location.get('city', 'N/A')
            state = location.get('state', 'N/A')
            country = location.get('country', 'N/A')
            categories = [category.get('title', 'N/A') for category in business.get('categories', [])]

            # Printing the business details
            print(f"Business Name: {name}")
            print(f"Rating: {rating}")
            print(f"Address: {address}, {city}, {state}, {country}")
            print(f"Categories: {', '.join(categories)}")
            print("-" * 50)
    else:
        print(f"No {category} businesses found in {location}.")
else:
    # Handle Rate Limit Exceeded
    if response.status_code == 429:
        print("Error: Rate limit exceeded. Please wait and try again later.")
        rate_limit_remaining = response.headers.get('RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")
    else:
        print(f"Error: {response.status_code}")
        print("Response Text:", response.text)


Business Name: Bangrak Market
Rating: 4.3
Address: 2319 2nd Ave, Seattle, WA, US
Categories: Bars, Thai, Tapas/Small Plates
--------------------------------------------------
Business Name: The Pink Door
Rating: 4.4
Address: 1919 Post Alley, Seattle, WA, US
Categories: Italian, Wine Bars, Pasta Shops
--------------------------------------------------
Business Name: Biscuit Bitch
Rating: 4.2
Address: 1909 1st Ave, Seattle, WA, US
Categories: Breakfast & Brunch, Coffee & Tea, Southern
--------------------------------------------------
Business Name: Reckless Noodle House
Rating: 4.5
Address: 2519 S Jackson St, Seattle, WA, US
Categories: Bars, Noodles, Asian Fusion
--------------------------------------------------
Business Name: Biang Biang Noodles
Rating: 4.3
Address: 601 E Pike St, Seattle, WA, US
Categories: Noodles, Szechuan
--------------------------------------------------
Business Name: Taku
Rating: 3.8
Address: 706 E Pike St, Seattle, WA, US
Categories: Izakaya, Bars
-----------

This will help us understand and analyze if their are events focused on Food as events are one such example were customers engage a lot and if more events happen in and around an area which means more customers and people are focused on those line of area

Query 6: Fetching reviews by different languages used by Customers

In [8]:
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Use 'application/json' for GraphQL
}

# 1. Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Seattle", limit: 10) {
    business {
      id
      name
      rating
      reviews {
        id
        rating
        text
        user {
          name
        }
      }
    }
  }
}
"""

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json={'query': query}, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()

    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print("Reviews:")

            # Process the reviews for each business
            for review in business.get('reviews', []):
                print(f"  Review ID: {review['id']}")
                print(f"  Rating: {review['rating']}")
                print(f"  Text: {review['text']}")
                print(f"  User: {review['user']['name']}")
                print("-" * 50)
    else:
        print("No restaurants found in Seattle.")
else:
    print(f"Error: {response.status_code}")
    print("Response Text:", response.text)


Business Name: The Pink Door
Rating: 4.4
Reviews:
  Review ID: 0aMqtTcYbTt2_KrlaP9WYQ
  Rating: 5
  Text: Service was great , and the vibe is really great too. It's busy, lively and romantic with their live music and interior decor. 

They serve great italian...
  User: Jimmy J.
--------------------------------------------------
  Review ID: eA_tcbKUl_X2H1umjFio4w
  Rating: 5
  Text: We always hit up the pink door whenever we visit Seattle! This last time my girlfriends and I shared two linguine with clams, once lasagna and the Caesar...
  User: Christine H.
--------------------------------------------------
  Review ID: D4yowh2QfXls79NxXLnEWA
  Rating: 4
  Text: Overall outstanding experience. Really 4.5 stars. The host and hostess, the wait staff, and the food were all fantastic. My wife and I were walk-ins, got...
  User: Jeff M.
--------------------------------------------------
Business Name: Von's 1000 Spirits
Rating: 4.5
Reviews:
  Review ID: 2cMKpQqSdT2Bk1GIxi_0BQ
  Rating: 5
 

This will give us idea on how customers use different languages to post their reviews and when we we open our restaurant then need to make sure we have language signs and guide cutomers and make them understand menu and our restaurant givings in different languages

Query 7:Here I am fetching top 10 customers and their details if available who have posted more than 100 reviews on Yelp.

In [13]:
import requests
import json
import time

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Correct header for GraphQL query in JSON format
}

# 1. Define parameters/data for the query
query = """
{
  search(term: "business", location: "Seattle", limit: 10) {
    total
    business {
      id
      name
      rating
      reviews {
        id
        rating
        text
        user {
          name

        }
      }
    }
  }
}
"""

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json={'query': query}, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print("Reviews:")

            # Process the reviews for each business
            for review in business.get('reviews', []):
                print(f"  Review ID: {review['id']}")
                print(f"  Rating: {review['rating']}")
                print(f"  Text: {review['text']}")
                print(f"  User: {review['user']['name']}")
                print("-" * 50)
    else:
        print("No businesses found in Seattle.")
else:
    if response.status_code == 429:
        print(f"Rate Limit Reached: {response.status_code}")
        rate_limit_remaining = response.headers.get('X-RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('X-RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")

        # Convert reset time (assuming it comes as a Unix timestamp) to local time
        if rate_limit_reset != 'Unknown':
            reset_time = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(int(rate_limit_reset)))
            print(f"Rate Limit Resets At: {reset_time} GMT")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)


Business Name: Eastside Trains
Rating: 3.8
Reviews:
  Review ID: cjTtF3nOSVJYBUUtv6fsfw
  Rating: 5
  Text: Nice to have a local train store with knowledgeable staff. They have always been helpful and friendly to my young son who loves Lionel.
  User: Eric B.
--------------------------------------------------
  Review ID: 7i8Dt0lLiE6F9AaEqYg8VA
  Rating: 4
  Text: Eastside has a great collection of larger scale equipment. I was impressed by the cases of N Scale coming through the door. I would have liked to see more...
  User: Karl A.
--------------------------------------------------
  Review ID: Ui450vDsxHxhCUbIsz6Apg
  Rating: 2
  Text: Clean and neat store but the downside is I did almost get attacked by their unleashed dog which I'm not cool with. 

It came zooming up and circling my...
  User: Piko L.
--------------------------------------------------
Business Name: Tonia's Mobile Notary
Rating: 5.0
Reviews:
  Review ID: QaxPKHD-WVNHsNWgBcKCqg
  Rating: 5
  Text: Came same day. V

This means they are frequent users of Yelp and understanding and analyzing their patterns can be very helpful to setup my new business venture and understand what customers want.

Query 8: Getting customers by their Yelp Join Dates and Years

In [9]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "reviewer", location: "Seattle", limit: 50) {
    total
    }
  }
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    users = data.get('data', {}).get('search', {}).get('user', [])

    if users:
        # Extract yelping_since (join year) and count occurrences
        join_years = [user['yelping_since'][:4] for user in users]  # Extract year (first 4 characters)
        year_counts = Counter(join_years)

        # Display the number of users joined each year
        for year, count in year_counts.items():
            print(f"Year: {year}, Users Joined: {count}")
    else:
        print("No users found.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)


No users found.


This will give us some some estimate to how many customers Yelp engages and are its users

Query 9: Query to know Icecream Parlours in Seattle area

In [12]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "ice cream", location: "Seattle", limit: 10) {
    total
    business {
      name
      location {
        address1
        city
        state
      }
      url
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        print("Restaurants serving ice cream in Seattle:")
        for business in businesses:
            print(f"Name: {business['name']}")
            print(f"Address: {business['location']['address1']}, {business['location']['city']}, {business['location']['state']}")
            print(f"URL: {business['url']}")
            print("-" * 50)
    else:
        print("No ice cream-serving restaurants found.")
else:
    print(f"Error: {response.status_code}")


Restaurants serving ice cream in Seattle:
Name: Molly Moon's Homemade Ice Cream
Address: 917 E Pine St, Seattle, WA
URL: https://www.yelp.com/biz/molly-moons-homemade-ice-cream-seattle-3?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Hellenika Cultured Creamery
Address: 1920A Pike Pl, Seattle, WA
URL: https://www.yelp.com/biz/hellenika-cultured-creamery-seattle?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Salt & Straw
Address: 714 E Pike St, Seattle, WA
URL: https://www.yelp.com/biz/salt-and-straw-seattle-2?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Lil' Tiger Ice Cream
Address: 12348 

Analyze number of Icecream parlours in the area and if they are less than Icecream can also be one division that I can setup in my restaurant

Query 10: Lastly lets fetch some business units across and their websites to further analyze and study

In [11]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "business", location: "Seattle", limit: 10) {
    business {
      name
      url
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        print("Business Units and their URLs in Seattle:")
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"URL: {business['url']}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Business Units and their URLs in Seattle:
Business Name: Eastside Trains
URL: https://www.yelp.com/biz/eastside-trains-kirkland-2?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: Tonia's Mobile Notary
URL: https://www.yelp.com/biz/tonias-mobile-notary-seattle-8?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: A Solution Repair Shop
URL: https://www.yelp.com/biz/a-solution-repair-shop-renton?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: The Foundry Clothing
URL: https://www.yelp.com/biz/the-foundry-clothing-seattle?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=ye

This will give us deep insights into business verticals and how they create their web portals to share information and market them selves in this competitive world

FINAL CONCLUSION AND INSIGHTS FROM THE ANALYSIS:


Exploring Opportunities for a New Restaurant Using Yelp GraphQL API:
  
The Yelp GraphQL API provided me access to detailed data on restaurants, reviews, locations, and customer preferences. By analysing this data, I aimed to identify key trends, gaps in the market, and potential concepts to consider for a new restaurant
Some insights:

1. Seattle's restaurant market offers a wide variety of cuisines, reflecting a diverse customer base.
2. There is a significant number of pizza-serving restaurants. While this indicates high demand, it also suggests market saturation. Differentiation is critical if you enter this niche.
3. Customers leave reviews in multiple languages, reflecting a multicultural customer base. Ensuring inclusivity through multilingual menus and staff could be an advantage.
4. Yelp super-users with more than 100 reviews represent a vocal and influential customer base. Attracting this group could amplify your restaurant's visibility.

A fusion restaurant blending Pacific Northwest ingredients (like seafood, wild mushrooms, and local produce) with global influences, especially Asian and Mediterranean flavors would cater to the adventurous food lovers who value sustainability, creativity, and local sourcing. Seattle's market has many single-cuisine restaurants (e.g., pizza, sushi). A fusion concept stands out.
A restauarant with **'Pacific Fusion: A Modern Take on Local and Global Flavors'** will stand out.
